In [363]:
from decouple import config
import gradio as gr
OPEN_ROUTER_API_KEY = config("OPEN_ROUTER_API_KEY")
GOOGLE_API_KEY = config("GOOGLE_API_KEY")
GOOGLE_GEMINI_MODEL1 = config("GOOGLE_GEMINI_MODEL1")
google_base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
openrouter_base_url = "https://openrouter.ai/api/v1"

In [364]:
from openai import OpenAI
client = OpenAI(
  base_url=google_base_url,
  api_key=GOOGLE_API_KEY,
)

In [365]:
system_prompt = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than one sentence.
Always be accurate. If you don't know the answer, say so.
"""


In [366]:
from enum import Enum
import logging
from logging.handlers import RotatingFileHandler
from pathlib import Path
from enum import Enum

class Handler(str,Enum):
    FILE = "fileHandler"
    STREAM = "streamHandler"
    ROTATING = "rotatingFileHandler"
    
def setUpLogger(*,dir_name = '', file_name = '', handler : Handler):
    if handler != Handler.STREAM:
    # dir and file setup
        dir_path = Path(dir_name)
        dir_path.mkdir(exist_ok=True)
        file_path = dir_path / f"{file_name}.log"
        if not file_path.exists():
            file_path.touch()
        
        # logger
    logger = logging.getLogger("mylogger")
    logger.setLevel(logging.DEBUG)
    logger.handlers.clear()
        
    formatter = logging.Formatter(
        fmt="{asctime} - {name} - {levelname} {filename}:{funcName}:{lineno} - {message}",
        style="{"
    )
    if handler == Handler.FILE:
        file_handler = logging.FileHandler(file_path)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    
    elif handler == Handler.STREAM:
        file_handler = logging.StreamHandler()
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
        
    elif handler == Handler.ROTATING:
        file_handler = RotatingFileHandler(file_path, maxBytes=10_000, backupCount=3)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    
    return logger

strlogger = setUpLogger(handler='streamHandler')
rotLogger = setUpLogger(dir_name='stream',file_name='app',handler='rotatingFileHandler')

In [367]:
ticket_prices = {
    "london": 799,
    "paris": 899,
    "tokyo": 1400,
    "berlin": 499
}
def get_ticket_price(destination_city : str) -> str:
    if destination_city.lower() not in ticket_prices:
        rotLogger.info(f"Price requested for unavailable city: {destination_city}")
        return f"Sorry, ticket prices for {destination_city} are not available"
    price = ticket_prices.get(destination_city.lower(),'')
    rotLogger.debug(f"Retrieved price for {destination_city}: ${price}")
    return f"The price of a ticket to {destination_city} is ${price}"

In [368]:
price_function = {
    "name" : "get_ticket_price",
    "description" : "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type" : "object",
        "properties" : {
            "destination_city" : {
                "type" : "string",
                "description" : "The city that the customer wants to travel to",
            },
        },
        "required" : ["destination_city"],
        "additionalProperties": False
    },
}

In [369]:
tools = [
    {
    "type" : "function",
    "function" : price_function
    },
    ]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [ ]:
from huggingface_hub import ChatCompletionOutputMessage
import json

def handle_tool_calling(message : ChatCompletionOutputMessage):
    tool = message.tool_calls[0]
    if tool.function.name == tools[0].get('function',{}).get('name',''):
        args = json.loads(tool.function.arguments)
        price_details  = get_ticket_price(**args)
        print(price_details)
        response = {
            'role' : 'tool',
            "content": price_details,
            'name': tool.function.name,
            "tool_call_id": tool.id,
        }
        return response

In [ ]:
from google.genai import types
google_tools = types.Tool(function_declarations=[price_function])
config = types.GenerateContentConfig(tools=[google_tools])

def call_llm_get_res(
        messages,
        model = "gemini-2.0-flash",
        tools=tools,
        tool_choice="auto"
    ):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=messages,
        config = config
    )
    return response

In [ ]:
messages = [{'role' : 'system', 'content' : system_prompt}]
def call_llm(message):
    messages.append({'role' : 'user', 'content' : message})
    response = call_llm_get_res(messages)
    output = response.choices[0].message.content
    if output:
        messages.append({'role' : 'assistant', 'content' : output})
        print(f"{output=}")
    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        response = handle_tool_calling(message)
        messages.append(response)
        print(messages)
        response = client.chat.completions.create(
            model="gemini-2.0-flash",
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        output = response.choices[0].message.content
        print(output)
    
call_llm("hello")    
call_llm("I want to go London.")    

output='Hello, how can I help you today?\n'
The price of a ticket to London is $799
[{'role': 'system', 'content': "\nYou are a helpful assistant for an Airline called FlightAI.\nGive short, courteous answers, no more than one sentence.\nAlways be accurate. If you don't know the answer, say so.\n"}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': 'Hello, how can I help you today?\n'}, {'role': 'user', 'content': 'I want to go London.'}, {'role': 'user2', 'content': 'The price of a ticket to London is $799', 'name': 'get_ticket_price', 'tool_call_id': 'function-call-15259706141316817697'}]


BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Invalid role: user2', 'status': 'INVALID_ARGUMENT'}}]